In [1]:
################################################################################
# File name:    "walking_distances.ipynb"
#
# Project title:    Boston Affordable Housing project (visting scholar porject)
#
# Description:    Calculates the walking distances (in meters) between closest
#                 properties to a boundary on either side of a boundary.
#
# Inputs:    ./effective_distance_inputs.csv
#
# Outputs:    ./effective_distance_outputs.csv
#
# Created:    10/17/2022
# Updated:    10/17/2022
#
# Author:    Nicholas Chiumenti
################################################################################

In [2]:
import os
import requests
import time
import random
import json
import pandas as pd
from datetime import datetime

In [3]:
start_time = datetime.now()
print("Running closest_boundary_no_roads program...")

Running closest_boundary_no_roads program...


# Load walking distance output dataset

In [5]:
# define path to input data
data_path = "/home/a1nfc04/Documents/boston_zoning_sdrive/data/walking_distance_inputs.csv"

# import as a dataframe
data_df = pd.read_csv(data_path)

# error checks
assert len(data_df) == 1358, "Incorrect number of observations in data_df."

print("Finished loading input data!")

Finished loading input data!


# Calculate walking/effective distance using osmr api

In [6]:
# Set Federal Reserve Bank of Boston proxies to get out of our firewall
os.environ['https_proxy']="p1web4.frb.org:8080"
os.environ['http_proxy']="p1web4.frb.org:8080"

# set headers for api
headers = {  "Accept":"application/json",
                'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
                "Accept-Encoding":"none",
                "Accept-Language":"en-US,en;q = 0.8",
                "Connection":"keep-alive",
                "Referer":"https://cssspritegenerator.com",
                "User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11"
                } 

# create empty list to store results
results = []

n = 0

for i, row in data_df.iterrows():

    boundary_id = row['boundary_using_id']
    
    # origin point
    lon_l = row['warren_longitudeLEFT']
    lat_l = row['warren_latitudeLEFT']
    
    # destination point
    lon_r = row['warren_longitudeRIGHT']
    lat_r = row['warren_latitudeRIGHT']

    # call osrm api
    url = (f"http://router.project-osrm.org/route/v1/foot/{lon_l},{lat_l};{lon_r},{lat_r}?overview=false")

    # unpack results
    r = requests.get(url, headers=headers)
    
    # unpack the response and store in dataframe
    if r.status_code == 200:

        output = json.loads(r.content)

        distance_m = output.get("routes")[0].get("distance")

        duration_s = output.get("routes")[0].get("duration")

        data_df.loc[i, "distance_m"] = distance_m
        data_df.loc[i, "duration_s"] = duration_s
        data_df.loc[i, "resp_code"] = r.status_code
    
    # if response code is not 200, store response code and move on
    else:
        data_df.loc[i, "resp_code"] = r.status_code
    
    # count observation 
    n += 1
    
    print(f"{n:,} of {len(data_df):,} observations completed", end="\r")

    # sleep before moving on
    time.sleep(random.randrange(1, 2))
    
# save data to final_df for export
final_df = data_df.copy()

print("Finished calculating walking distances!")

Finished calculating walking distances!


# Export data and save a log

In [7]:
final_df = data_df.copy()
# set save and log paths
save_path = f"/home/a1nfc04/Documents/boston_zoning_sdrive/data/walking_distance_outputs.csv"

log_path = "/home/a1nfc04/Documents/boston_zoning_sdrive/python_programs/walking_distances/walking_distances_orsm_log.txt"

# create log and save date stamps
end_time = datetime.now()

duration = end_time - start_time

duration_in_s = (duration.days * 24 * 60 * 60) + duration.seconds
mins, secs = divmod(duration_in_s, 60)
hours, mins = divmod(mins, 60)
days, hours  = divmod(hours, 24)

# save dataset as .csv
final_df.to_csv(save_path, index = False)

# write to log
with open(log_path,'a') as file:
    file.write(f"Last run on {datetime.now().strftime('%D at %I:%M:%S %p')}\n")
    file.write(f"{len(final_df):,} observations written to {save_path} \n")
    file.write(f"Total run time: {days} days, {hours:02} hours, {mins:02} minutes, {secs:02} seconds \n\n")

# Done!
print(f"Done, {len(final_df):,} observations written!")

Done, 1,358 observations written!
